In [2]:

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup ..

In [3]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [4]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [5]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, DmGQBVp5TUUjdB4bWCUps8VbuYZuMrHUmumAfSWEuwMz, 3>


In [6]:

display(print(my_graph.num_nodes()))
display(print(my_graph.num_edges()))


          0/? [?op/s]

4


None

          0/? [?op/s]

7


None

In [ ]:

#  What's in the vertices/nodes ..

l_query  = """
   MATCH ( n ) 
   RETURN n
   """.format()

l_result = my_graph.query(l_query)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

#  What's in the edge .. 

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN r 
   """.format()

l_result = my_graph.query(l_query)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """.format()

l_result = my_graph.query(l_query, contextualize=True)

l_result.view()


# Actual REST ..

In [7]:

#  This used to work from the Jupyter VM,
#     curl localhost:8080/v2/version
#
#  Now, the above only works from the Controller, or if we
#  target the controller


# Boilerplate ..

import http.client
import json

#  l_conn   = http.client.HTTPConnection("localhost", 8080)
l_conn   = http.client.HTTPConnection("10.0.0.65", 8080)                           #  Cheat; look this up manually

l_graphid = my_graph.graph_id
   #
print("Graph Id: " + l_graphid)


Graph Id: DmGQBVp5TUUjdB4bWCUps8VbuYZuMrHUmumAfSWEuwMz


In [10]:

#  Submit request ..

l_body   = json.dumps({
  "Cypher": {
    "query": "MATCH (n) RETURN n",
    "output_type": "JSON"
  }
})
l_header = {
  'Content-Type': 'application/json'
}


l_conn.request("POST", "/v2/graph/" + l_graphid + "/run", l_body, l_header)


l_response   = l_conn.getresponse()
l_data       = l_response.read()

   # 

#  l_data_dict  = json.loads((l_data.decode("utf-8")))
#  l_opid       = (l_data_dict["operation_id"])

#  print(l_opid)


print(l_data)





ResponseNotReady: Request-sent

In [ ]:

#  Get status ..

l_conn.request("GET", "/v2/operation/" + l_opid + "/status")
   #
l_response   = l_conn.getresponse()
l_data       = l_response.read()

l_data_dict  = json.loads((l_data.decode("utf-8")))

print(l_data_dict["status"])


In [ ]:

#  Get data ..

l_conn.request("GET", "/v2/operation/" + l_opid)
   #
l_response   = l_conn.getresponse()
l_data       = l_response.read()

l_data_dict  = json.loads((l_data.decode("utf-8")))

print(l_data_dict["result"]["rows"])


#  Full result ..
#
#  {
#  'operation_id': '3mHuoyjZt74SicqWfzHfjxSSHGnD8fTVdCM3MZaVkpFF-R7s2MLsdnhhAzzvy',
#  'status': 'Complete',
#  'status_message': '', 
#  'status_modified_at': '2022-08-24T15:16:54.229182259Z', 
#  'operation': {
#     'cypher': {'query': 'MATCH (n) RETURN n', 'parameters': None, 'output_type': 'JSON'}
#      }, 
#  'result': {
#     'column_types': ['struct<type: large_string, id: uint32, labels: large_list<item: large_string>, properties: struct<LABEL: large_string, airport_name: large_string, id: large_string>>'], 
#     'columns': ['n'], 
#     'num_rows': 4, 
#     'rows': [
#         {'n': {'id': 0, 'labels': ['Airport'], 'properties': {'LABEL': 'Airport', 'airport_name': 'Milwaukee', 'id': 'MKE'}, 'type': 'node'}}, 
#         {'n': {'id': 2, 'labels': ['Airport'], 'properties': {'LABEL': 'Airport', 'airport_name': 'San Jose', 'id': 'SJC'}, 'type': 'node'}}, 
#         {'n': {'id': 3, 'labels': ['Airport'], 'properties': {'LABEL': 'Airport', 'airport_name': 'Denver', 'id': 'DEN'}, 'type': 'node'}}, 
#         {'n': {'id': 1, 'labels': ['Airport'], 'properties': {'LABEL': 'Airport', 'airport_name': 'Chicago O-Hare', 'id': 'ORD'}, 'type': 'node'}}
#        ]}}
